In [118]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

[nltk_data] Downloading package words to
[nltk_data]     /Users/catherinegrimes/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [119]:
df_billboard = pd.read_csv('data/Hot 100.csv')

In [120]:
df_tracks = pd.read_csv('data/tracks.csv')

In [121]:
df_artists = pd.read_csv('data/artists.csv')

In [122]:
df_tracks.fillna('NAN', inplace = True)

In [123]:
#clean

df_billboard['performer'] = [utils.default_process(performer) for performer in df_billboard.performer]
df_billboard['song'] = [utils.default_process(song) for song in df_billboard.song]
df_tracks['artists'] = [utils.default_process(artist) for artist in df_tracks.artists]
df_tracks['name'] = [utils.default_process(name) for name in df_tracks.name]
df_tracks['id_artists'] = [re.sub(r'[^0-9a-zA-Z]', '', name) for name in df_tracks.id_artists]

In [124]:
#clean genres column, 40 most common, pick first from list as the singular genre

df_artists['genres'] = [re.sub(r'[^\w\s,]', '', genre) for genre in df_artists['genres']]
df_artists['genres'] = [genre.split(',') for genre in df_artists['genres']]
max_length = max(len(sublist) for sublist in df_artists['genres'])
padded_list = [sublist + [None] * (max_length - len(sublist)) for sublist in df_artists['genres']]
flattened_padded_list = [item for sublist in padded_list for item in sublist if item is not None]

classes, counts = np.unique(flattened_padded_list, return_counts=True)


element_count_pairs = list(zip(classes, counts))
element_count_pairs.sort(reverse=True, key=lambda x: x[1])

top_genres = [element for element, count in element_count_pairs[:41]]
top_genres = top_genres[1:]

df_artists = df_artists[[any(item in top_genres for item in row) for row in df_artists['genres']]]
df_artists['genres'] = [[item for item in row if item in top_genres] for row in df_artists['genres']]
df_artists['genre'] = [x[0] for x in df_artists['genres']]

In [125]:
#remove extra white space

df_tracks['artists'] = [' '.join(artist.split()) for artist in df_tracks.artists]
df_tracks['name'] = [' '.join(name.split()) for name in df_tracks.name]
df_billboard['performer'] = [' '.join(performer.split()) for performer in df_billboard.performer]
df_billboard['song'] = [' '.join(song.split()) for song in df_billboard.song]
df_artists['genre'] = [' '.join(genre.split()) for genre in df_artists.genre]

In [126]:
#specific problems

df_billboard['performer'] = [re.sub(r'Sylvia\s\(r&b\)', 'Sylvia', performer) for performer 
                             in df_billboard.performer]
df_billboard['performer'] = [re.sub(r'\(The Preacher\)\sBobby Womack', 'Bobby Womack', performer) for performer 
                             in df_billboard.performer]
df_tracks['name'] = [re.sub(r'bei\smir\sbist\sdu\sschoen', 'bei mir bist du schön', name) 
                     for name in df_tracks.name]
df_tracks['name'] = [re.sub(r'bei\smir\sbist\sdu\sschon', 'bei mir bist du schön', name) 
                     for name in df_tracks.name]
df_tracks['name'] = [re.sub(r'bei\smir\sbist\sdu\sscheen', 'bei mir bist du schön', name) 
                     for name in df_tracks.name]
df_tracks['artists'] = [re.sub(r'p\snk', 'pink', artist) for artist in df_tracks.artists]
df_billboard['performer'] = [re.sub(r'p\snk', 'pink', performer) for performer in df_billboard.performer]

In [127]:
#multiple artists

df_tracks['artists'] = [re.sub(r'featuring', '', artist) for artist in df_tracks.artists]
df_tracks['artists'] = [re.sub(r'\sfeat(?!\s*$)', '', artist) for artist in df_tracks.artists]
df_tracks['artists'] = [re.sub(r'with\s', '', artist) for artist in df_tracks.artists]
df_billboard['performer'] = [re.sub(r'featuring', '', performer) for performer in df_billboard.performer]
df_billboard['performer'] = [re.sub(r'with\s', '', performer) for performer in df_billboard.performer]

In [128]:
#condense billboard data: only until first drop off the charts, find row of peak position

df_billboard = df_billboard[df_billboard['instance'] == 1]
df_billboard['time_on_chart'] = df_billboard.groupby('song_id')['time_on_chart'].transform('max')
df_billboard = df_billboard.loc[df_billboard.groupby('song_id')['peak_position'].idxmin()]

In [129]:
df = df_tracks.merge(df_billboard, how = 'left', left_on = ['name', 'artists'], right_on = ['song', 'performer'])

In [130]:
df = df.merge(df_artists, how = 'left', left_on = 'id_artists', right_on = 'id')

In [131]:
df = df.dropna(subset = ['genre'])

In [132]:
#drop rows without exact release date, chop up into day and month

df = df[df['release_date'].str.len() == 10]
df['peak_date'] = pd.to_datetime(df['chart_date'])
df['chart_debut'] = pd.to_datetime(df['chart_debut'])
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_month'] = df['release_date'].dt.month
df['release_day'] = df['release_date'].dt.day
df['no_year'] = df['release_date'].dt.strftime('%m-%d')
df['year'] = df['release_date'].dt.year

In [133]:
#remove re-releases: take earliest release

df = df.loc[df.groupby(['name_x', 'artists'])['release_date'].idxmin()]

In [134]:
#create season variable

df['season'] = pd.cut(x = df['release_month'], bins = [1, 5, 8, 11], labels = ['Spring', 'Summer', 'Fall'])
df['season'] = df['season'].cat.add_categories('Winter')
df['season'] = df['season'].fillna('Winter')

In [135]:
#create day of week variable for release date

df['weekday'] = [date.weekday() for date in df['release_date']]
df['weekday'] = pd.cut(x = df['weekday'], bins = 7, labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
                                                                 'Friday', 'Saturday', 'Sunday'])

In [136]:
#binary for charting

df['charted'] = np.where(df['chart_debut'].isna(), 0, 1)

In [137]:
#organizing

df = df.drop(columns = ['explicit', 'chart_url', 'previous_week', 'time_signature', 'liveness', 'speechiness', 
                        'mode','loudness', 'key', 'instance', 'acousticness', 'artists', 
                        'popularity_x','instrumentalness', 'tempo', 'id_x', 'id_y', 'id_artists', 'name_x', 
                        'performer', 'danceability', 'followers', 'consecutive_weeks', 'song_id', 'chart_position',
                       'genres', 'worst_position'])
df = df.rename(columns = {'name_y': 'artist', 'popularity_y': 'superstar', 'time_on_chart': 'weeks_on_chart'})
df = pd.concat([df[['song', 'artist', 'release_date', 'charted']], 
                df.drop(['song', 'artist', 'release_date', 'charted'], axis = 1)], axis = 1)

In [138]:
df['weeks_on_chart'] = np.where(df['weeks_on_chart'].isna(), 0, df['weeks_on_chart'])
df['song'] = np.where(df['song'].isna(), 'symbol', df['song'])
df['chart_date'].fillna('NAN', inplace = True)

In [139]:
df.to_csv('working.csv')